# 필요한 라이브러리 불러오기

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import joblib

import re
import json

from konlpy.tag import Mecab
import nltk

from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.models import Word2Vec, Phrases

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

### 블용어 지정

In [2]:
stop_words = "function,flashremoveCallback,오류,우회,하기,함수,추가,포토,기자,증권,뉴스,무단,배포,금지,전재 "
stop_words = stop_words.split(',')

### 뉴스기사 전처리를 위한 함수

In [3]:
# 문서의 토큰화를 위해서 한국어만 남겨둠
def text_clean(doc):
    doc =re.sub("[^a-zA-Zㄱ-ㅎㅏ-ㅣ가-힣▲△↑▼▽↓ ]", "", doc)
    return doc

# 문서를 명사로 토큰화
def noun_tokenizer(text):
    nouns = Mecab().nouns(text)
    nouns = [noun for noun in nouns if noun not in stop_words]
    return [noun for noun in nouns if (len(noun) > 1)]

# 뉴스 기사 불러오기(증권 네이버뉴스 전처리 4개년)

In [4]:
df_증권뉴스=pd.read_csv('./증권4개년 뉴스 전처리.csv')
df_증권뉴스.head(20)

date                                       제목                 작성시간  \
0   2017-01-01                         인사 국민연금공단 기금운용본부  2017-01-01 22:53:00   
1   2017-01-01                  국민연금 기금운용본부 운용전략실장에 이수철  2017-01-01 21:37:00   
2   2017-01-01                  금리 오르면 사모펀드에 투자기회 늘어날 것  2017-01-01 19:45:00   
3   2017-01-01                 마켓인사이트 KB증권 JW바이오 지분  인수  2017-01-01 19:41:00   
4   2017-01-01                증권사 이달부터 ELS 헤지자산 구분 관리해야  2017-01-01 19:39:00   
5   2017-01-01                  국민연금 기금운용본부 운용전략실장에 이수철  2017-01-01 19:39:00   
6   2017-01-01                 상하이증시 자본유출통상전쟁 등 악재 첩첩산중  2017-01-01 19:32:00   
7   2017-01-01                  뉴욕증시 월 효과 기대고용제조업 지표 관심  2017-01-01 19:31:00   
8   2017-01-01                     한일퍼스트운용 헤지펀드 운용사로 출범  2017-01-01 19:31:00   
9   2017-01-01            황영기 야성상상력으로 혁신박현주 익숙한 것과 결별하라  2017-01-01 19:29:00   
10  2017-01-01             지난달 서울 아파트 거래량 급감전달에 비해 나 줄어  2017-01-01 19:21:00   
11  2017-01-01                 국민연금 업무 마비국민 불신 커질까 더 걱정  2017-01-01 18:35:00   
12  2017-01-01                 주간 브리핑 새해 첫 증시 일 오전 시 개장  2017-01-01 18:29:00   
13  2017-01-01          IT 주도로 박스피 뚫을 것  금리인상에도 증시전망 밝아  2017-01-01 18:25:00   
14  2017-01-01          대예측  원유 오르고 금 내린다 원자재펀드도 옥석 가려라  2017-01-01 18:09:00   
15  2017-01-01          대예측  올해 코스피 전망 pt 박스피 뚫고 지붕킥 할까  2017-01-01 18:08:00   
16  2017-01-01                   대전환 골든타임  금융허브로 떠오른 선전  2017-01-01 17:53:00   
17  2017-01-01   대전환 골든타임 증시 돈맥경화 선진국보다 높은 투자 문턱 낮춰 풀어야  2017-01-01 17:53:00   
18  2017-01-01      대도약   중반 성장 사수 소비심리 개선에 달려수출은 회복 기대  2017-01-01 17:42:00   
19  2017-01-01                   대도약  당분간 강달러분기 원 찍을 수도  2017-01-01 17:39:00   

                                                  형태소  점수      date.1  
0          ['인사', '국민', '연금', '공단', '기금', '운용', '본부']   0  2017-01-01  
1   ['국민', '연금', '기금', '운용', '본부', '운용', '전략', '실장...   0  2017-01-01  
2   ['금리', '오르', '면', '사모', '펀드', '에', '투자', '기회',...   1  2017-01-01  
3   ['마켓', '인사이트', 'KB', '증권', 'JW', '바이오', '지분', ...   1  2017-01-01  
4   ['증권사', '이달', '부터', 'ELS', '헤지', '자산', '구분', '...   0  2017-01-01  
5   ['국민', '연금', '기금', '운용', '본부', '운용', '전략', '실장...   0  2017-01-01  
6   ['상하', '이증', '시', '자본', '유출', '통상', '전쟁', '등',...  -1  2017-01-01  
7   ['뉴욕', '증시', '월', '효과', '기대', '고용', '제조업', '지표...   0  2017-01-01  
8   ['한', '일', '퍼스트', '운용', '헤지', '펀드', '운용', '사',...   0  2017-01-01  
9   ['황영기', '야성', '상상력', '으로', '혁신', '박현주', '익숙', ...  -1  2017-01-01  
10  ['지난달', '서울', '아파트', '거래량', '급감', '전달', '에', '...  -1  2017-01-01  
11  ['국민', '연금', '업무', '마비', '국민', '불신', '커', '질까'...   0  2017-01-01  
12  ['주간', '브리핑', '새해', '첫', '증시', '일', '오전', '시',...   0  2017-01-01  
13  ['IT', '주도', '로', '박스', '피', '뚫', '을', '것', '금...   1  2017-01-01  
14  ['대', '예측', '원유', '오르', '고', '금', '내린다', '원자재'...   0  2017-01-01  
15  ['대', '예측', '올해', '코스피', '전망', 'pt', '박스', '피'...   1  2017-01-01  
16  ['대전환', '골든', '타임', '금융', '허브', '로', '떠오른', '선전']   0  2017-01-01  
17  ['대전환', '골든', '타임', '증시', '돈', '맥경', '화', '선진국...   1  2017-01-01  
18  ['대', '도약', '중반', '성장', '사수', '소비', '심리', '개선'...   1  2017-01-01  
19  ['대', '도약', '당분간', '강', '달러', '분기', '원', '찍', ...   0  2017-01-01

In [5]:
df_증권뉴스.isnull().sum()

date      0
제목        0
작성시간      0
형태소       0
점수        0
date.1    0
dtype: int64

# TfidfVectorizer를 이용해 벡터화

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
# tf_vect = TfidfVectorizer(ngram_range=(1,2), min_df = round(len(df_증권뉴스)/3))
tf_vect = TfidfVectorizer(tokenizer = noun_tokenizer, \
                         ngram_range=(1,2))

In [7]:
# feature_vect=tf_vect.fit_transform(df_증권뉴스['형태소'].apply(lambda x: ' '.join(x)))
feature_vect=tf_vect.fit_transform(df_증권뉴스['제목'])

# KMeans를 이용해 군집화

In [9]:
from sklearn.cluster import KMeans

km_cluster = KMeans(n_clusters=3, max_iter=100, random_state=0)
km_cluster.fit(feature_vect)
cluster_label = km_cluster.labels_
cluster_centers = km_cluster.cluster_centers_

In [10]:
df_증권뉴스['cluster_label']=cluster_label
df_증권뉴스.head()

date                         제목                 작성시간  \
0  2017-01-01           인사 국민연금공단 기금운용본부  2017-01-01 22:53:00   
1  2017-01-01    국민연금 기금운용본부 운용전략실장에 이수철  2017-01-01 21:37:00   
2  2017-01-01    금리 오르면 사모펀드에 투자기회 늘어날 것  2017-01-01 19:45:00   
3  2017-01-01   마켓인사이트 KB증권 JW바이오 지분  인수  2017-01-01 19:41:00   
4  2017-01-01  증권사 이달부터 ELS 헤지자산 구분 관리해야  2017-01-01 19:39:00   

                                                 형태소  점수      date.1  \
0         ['인사', '국민', '연금', '공단', '기금', '운용', '본부']   0  2017-01-01   
1  ['국민', '연금', '기금', '운용', '본부', '운용', '전략', '실장...   0  2017-01-01   
2  ['금리', '오르', '면', '사모', '펀드', '에', '투자', '기회',...   1  2017-01-01   
3  ['마켓', '인사이트', 'KB', '증권', 'JW', '바이오', '지분', ...   1  2017-01-01   
4  ['증권사', '이달', '부터', 'ELS', '헤지', '자산', '구분', '...   0  2017-01-01   

   cluster_label  
0              1  
1              1  
2              1  
3              1  
4              1

In [11]:
df_증권뉴스[df_증권뉴스['cluster_label']==0]

date                        제목                 작성시간  \
406      2017-01-02      fnRASSI국일신동 전일대비  상승  2017-01-02 15:25:00   
407      2017-01-02     fnRASSI미래컴퍼니 전일대비  상승  2017-01-02 15:25:00   
454      2017-01-02   fnRASSI마이크로프랜드 전일대비  상승  2017-01-02 15:14:00   
456      2017-01-02       fnRASSI옵토팩 전일대비  상승  2017-01-02 15:13:00   
466      2017-01-02        fnRASSI인콘 전일대비  상승  2017-01-02 15:10:00   
...             ...                       ...                  ...   
1361585  2020-08-26       코스피 전일대비 p 오른  출발 ↑  2020-08-26 09:02:00   
1423105  2020-11-11   Torm Q EBITDA  Mln으로 상승  2020-11-11 00:00:00   
1449440  2020-12-15             코스피 전일 대비  하락  2020-12-15 16:17:00   
1449453  2020-12-15  코스피 전일대비 하락 환율은 소폭 상승 마감  2020-12-15 16:10:00   
1457711  2020-12-24       속보 코스피 전일대비  오른  출발  2020-12-24 09:01:00   

                                                       형태소  점수      date.1  \
406          ['fnRASSI', '국', '일', '신동', '전일', '대비', '상승']   1  2017-01-02   
407             ['fnRASSI', '미래', '컴퍼니', '전일', '대비', '상승']   1  2017-01-02   
454       ['fnRASSI', '마이크', '로프', '랜드', '전일', '대비', '상승']   1  2017-01-02   
456           ['fnRASSI', '옵', '토', '팩', '전일', '대비', '상승']   1  2017-01-02   
466                ['fnRASSI', '인', '콘', '전일', '대비', '상승']   1  2017-01-02   
...                                                    ...  ..         ...   
1361585          ['코스피', '전일', '대비', 'p', '오른', '출발', '↑']   1  2020-08-26   
1423105         ['Torm', 'Q', 'EBITDA', 'Mln', '으로', '상승']   1  2020-11-11   
1449440                          ['코스피', '전일', '대비', '하락']  -1  2020-12-15   
1449453  ['코스피', '전일', '대비', '하락', '환율', '은', '소폭', '상승...   0  2020-12-15   
1457711              ['속보', '코스피', '전일', '대비', '오른', '출발']   1  2020-12-24   

         cluster_label  
406                  0  
407                  0  
454                  0  
456                  0  
466                  0  
...                ...  
1361585              0  
1423105              0  
1449440              0  
1449453              0  
1457711              0  

[87127 rows x 7 columns]

In [12]:
df_증권뉴스[df_증권뉴스['cluster_label']==1]

date                                 제목                 작성시간  \
0        2017-01-01                   인사 국민연금공단 기금운용본부  2017-01-01 22:53:00   
1        2017-01-01            국민연금 기금운용본부 운용전략실장에 이수철  2017-01-01 21:37:00   
2        2017-01-01            금리 오르면 사모펀드에 투자기회 늘어날 것  2017-01-01 19:45:00   
3        2017-01-01           마켓인사이트 KB증권 JW바이오 지분  인수  2017-01-01 19:41:00   
4        2017-01-01          증권사 이달부터 ELS 헤지자산 구분 관리해야  2017-01-01 19:39:00   
...             ...                                ...                  ...   
1461866  2020-12-31         한국항공우주 국방과학연구소와 억 규모 계약 체결  2020-12-31 13:37:00   
1461867  2020-12-31         사라지지 않은 코로나백신치료제 더 오른다 유망주  2020-12-31 13:34:00   
1461868  2020-12-31  동학개미 덕에 두 배 뛴 반도체슈퍼사이클로 더 간다  유망주  2020-12-31 13:34:00   
1461869  2020-12-31             채권 전문가  내년 채권 금리 보합 전망  2020-12-31 13:33:00   
1461870  2020-12-31                           인사KR투자증권  2020-12-31 13:29:00   

                                                       형태소  점수      date.1  \
0               ['인사', '국민', '연금', '공단', '기금', '운용', '본부']   0  2017-01-01   
1        ['국민', '연금', '기금', '운용', '본부', '운용', '전략', '실장...   0  2017-01-01   
2        ['금리', '오르', '면', '사모', '펀드', '에', '투자', '기회',...   1  2017-01-01   
3        ['마켓', '인사이트', 'KB', '증권', 'JW', '바이오', '지분', ...   1  2017-01-01   
4        ['증권사', '이달', '부터', 'ELS', '헤지', '자산', '구분', '...   0  2017-01-01   
...                                                    ...  ..         ...   
1461866  ['한국', '항공우주', '국방과학연구소', '와', '억', '규모', '계약'...   1  2020-12-31   
1461867  ['사라지', '지', '않', '은', '코로나', '백신', '치료제', '더'...   1  2020-12-31   
1461868  ['동학', '개미', '덕', '에', '두', '배', '뛴', '반도체', '...   1  2020-12-31   
1461869        ['채권', '전문가', '내년', '채권', '금리', '보합', '전망']   0  2020-12-31   
1461870                           ['인사', 'KR', '투자', '증권']   0  2020-12-31   

         cluster_label  
0                    1  
1                    1  
2                    1  
3                    1  
4                    1  
...                ...  
1461866              1  
1461867              1  
1461868              1  
1461869              1  
1461870              1  

[1249621 rows x 7 columns]

In [13]:
df_증권뉴스[df_증권뉴스['cluster_label']==2]

date                          제목                 작성시간  \
114      2017-01-01  마켓포인트코스피 주간 외국인 순매수 위 삼성전자  2017-01-01 12:02:00   
225      2017-01-02          표코스피 기관외국인 매매동향 시분  2017-01-02 16:36:00   
232      2017-01-02         코스피 기관 매도세에 약보합  마감  2017-01-02 16:33:00   
243      2017-01-02                  코스피 약보합 마감  2017-01-02 16:23:00   
244      2017-01-02                첫 거래일 코스피 하락  2017-01-02 16:23:00   
...             ...                         ...                  ...   
1461627  2020-12-31              e 공시 눈에 띄네코스피일  2020-12-31 07:10:00   
1461632  2020-12-31         재송일 장 마감 후 주요 종목 뉴스  2020-12-31 07:00:00   
1461643  2020-12-31          코스피코스닥 전 거래일일 주요공시  2020-12-31 06:05:00   
1461690  2020-12-31  삼성전자 만전자 시대 코스피  사상 최고치 마감  2020-12-31 03:04:00   
1461739  2020-12-31          표코스피 기관외국인 매매동향 시분  2020-12-31 17:00:00   

                                                       형태소  점수      date.1  \
114      ['마켓', '포인트', '코스피', '주간', '외국인', '순', '매수', '...   1  2017-01-01   
225            ['표', '코스피', '기관', '외국인', '매매', '동향', '시분']   0  2017-01-02   
232                 ['코스피', '기관', '매도', '세에', '약보합', '마감']  -1  2017-01-02   
243                                   ['코스피', '약보합', '마감']   0  2017-01-02   
244                              ['첫', '거래일', '코스피', '하락']  -1  2017-01-02   
...                                                    ...  ..         ...   
1461627        ['e', '공시', '눈', '에', '띄', '네', '코스피', '일']   0  2020-12-31   
1461632      ['재송', '일', '장', '마감', '후', '주요', '종목', '뉴스']   0  2020-12-31   
1461643        ['코스피', '코스닥', '전', '거래일', '일', '주요', '공시']   0  2020-12-31   
1461690  ['삼성전자', '만', '전자', '시대', '코스피', '사상', '최고', '...   1  2020-12-31   
1461739        ['표', '코스피', '기관', '외국인', '매매', '동향', '시분']   0  2020-12-31   

         cluster_label  
114                  2  
225                  2  
232                  2  
243                  2  
244                  2  
...                ...  
1461627              2  
1461632              2  
1461643              2  
1461690              2  
1461739              2  

[125123 rows x 7 columns]

In [14]:
# argsort는 정열된 값의 인덱스를 출력
top_index=cluster_centers.argsort()[:,::-1][:,:20]

In [15]:
feature_names=tf_vect.get_feature_names()

In [16]:
top_index[0].tolist()

[839873,
 839856,
 211139,
 211397,
 471412,
 211786,
 1131138,
 473629,
 1213454,
 1213447,
 144802,
 53128,
 665843,
 35234,
 582328,
 473008,
 52625,
 35078,
 1220403,
 665645]

In [17]:
# 군집1의 키워드
np.array(feature_names)[top_index[0].tolist()]

array(['전일 대비', '전일', '대비', '대비 상승', '상승', '대비 하락', '하락', '상승 회전율',
       '회전율 기록', '회전율', '기록', '경신 전일', '연속 상승', '거래일 연속', '신고 경신',
       '상승 전일', '경신', '거래일', '흐름', '연속'], dtype='<U17')

In [18]:
# 군집2의 키워드
np.array(feature_names)[top_index[1].tolist()]

array(['투자', '분기', '상승', '영업', '코스닥', '매수', '신고', '이상', '증시', '이상 상승',
       '전년', '증가', '규모', '실적', '외국인', '특징', '종목', '주가', '거래량', '결정'],
      dtype='<U17')

In [19]:
# 군집3의 키워드
np.array(feature_names)[top_index[2].tolist()]

array(['코스피', '마감', '코스닥', '매수', '출발', '환율', '하락', '상승', '우위', '마감 코스피',
       '달러 환율', '강세 업종', '매도', '업종', '코스피 상승', '매수 우위', '강세', '시황', '외국인',
       '달러'], dtype='<U17')

# cosine_similarity를 이용해 유사도 파악

In [1]:
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
# center point vector와 개별 데이터 vector들과의 유사도 계산
center_sim = cosine_similarity(cluster_centers, feature_vect.toarray())
center_sim

NameError: name 'cluster_centers' is not defined

In [ ]:
# 0번 클러스터의 center point vector와 유사도가 높은 데이터 vector들을 출력

In [ ]:
clu_0_sim_top10_idx = center_sim[0].argsort()[::-1][:10]
clu_0_sim_top10_idx

In [ ]:
document_df.filename[clu_0_sim_top10_idx]

In [ ]:
# 1번 클러스터의 center point vector와 유사도가 높은 데이터 vector들을 출력
clu_1_sim_top10_idx = center_sim[1].argsort()[::-1][:10]
document_df.filename[clu_1_sim_top10_idx]

In [ ]:
# 2번 클러스터의 center point vector와 유사도가 높은 데이터 vector들을 출력
clu_2_sim_top10_idx = center_sim[2].argsort()[::-1][:10]
document_df.filename[clu_2_sim_top10_idx]